## **TABLA DE MODELADO**

In [1]:
# DATOS
import pandas as pd

# BASE DE DATOS
from sqlalchemy import create_engine

# MODELADO
from joblib import load

**MODELOS**

In [4]:
# KMEANS
kmeans = load('C:/Users/20391117579/Dropbox/CrimeApp/Data Science Lab/Modelos/kmeans.joblib')
scaler = load('C:/Users/20391117579/Dropbox/CrimeApp/Data Science Lab/Modelos/scaler.joblib')

**TABLA FILTRADA DE SQL**

In [5]:
hostname = "localhost"
dbname = "crimewarehouse"
uname = "root"
pwd = "admin1234"

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=hostname, db=dbname, user=uname, pw=pwd))

In [6]:
def get_fct_table(query, db):
    df_fct = pd.read_sql(query, db)

    return df_fct

df_fct = get_fct_table(query="SELECT * FROM FCT_HECHOS;", db=engine)

In [7]:
df_fct.head(5)

,CONTACTO_ID,FECHA,FRANJA_HORARIA,TIPO_DELITO_KEY,LATITUD,LONGITUD,COMUNA_KEY,BARRIO_KEY
0,0,2016-01-31,12,1,-34.6398,-58.5297,9,22
1,1,2016-01-31,8,1,-34.6388,-58.5275,9,22
2,2,2016-01-31,20,1,-34.6456,-58.5265,9,22
3,3,2016-01-31,22,1,-34.6013,-58.5199,11,31
4,4,2016-01-31,8,1,-34.5896,-58.5187,11,31


**TABLA DE MODELADO**

In [8]:
def transform_time_series(df_fct):
    # Serie temporal
    df_fct['FECHA'] = pd.to_datetime(df_fct['FECHA'])
    df_fct['MES'] = df_fct['FECHA'].dt.month 
    df_fct['DIA_DEL_MES'] = df_fct['FECHA'].dt.day  
    df_fct['DIA_DEL_AÑO'] = df_fct['FECHA'].dt.day_of_year
    df_fct['SEMANA_DEL_AÑO'] = df_fct['FECHA'].dt.isocalendar().week.astype(int)
    df_fct['TRIMESTRE'] = df_fct['FECHA'].dt.quarter  
    df_fct['DIA_DE_LA_SEMANA'] = df_fct['FECHA'].dt.dayofweek 

    # Estacion, Fin de semana
    df_fct['ES_FIN_DE_SEMANA'] = df_fct['DIA_DE_LA_SEMANA'].apply(lambda x: 1 if x in [5, 6] else 0)
    df_fct['ESTACION'] = df_fct['MES'].apply(lambda x: 4 if x in [12, 1, 2] else (1 if x in [3, 4, 5] else (2 if x in [6, 7, 8] else 3)))

    # Horario Categorizado
    bins = [0, 6, 12, 18, 24]
    labels = [1, 2, 3, 4]
    df_fct['HORARIO_CATEGORIZADO'] = pd.cut(df_fct['FRANJA_HORARIA'], bins=bins, labels=labels, right=False).astype(int)

    df_fct_time_series = df_fct[['FECHA', 'LATITUD', 'LONGITUD', 'ESTACION', 'MES', 'TRIMESTRE',
                                'SEMANA_DEL_AÑO', 'DIA_DEL_AÑO', 'DIA_DEL_MES', 'DIA_DE_LA_SEMANA',
                                'ES_FIN_DE_SEMANA', 'HORARIO_CATEGORIZADO', 'FRANJA_HORARIA','COMUNA_KEY', 'BARRIO_KEY', 'TIPO_DELITO_KEY']]

    return df_fct_time_series

In [9]:
df_fct_time_series = transform_time_series(df_fct)

In [16]:
def transform_location(df_fct_time_series, kmeans, scaler):
    # Zona
    coordenadas = df_fct_time_series[['LATITUD','LONGITUD']]
    x_scaled = scaler.fit_transform(coordenadas)

    df_fct_time_series['CLUSTER'] = kmeans.predict(x_scaled)
    df_fct_time_series['ZONA'] = df_fct_time_series['CLUSTER'] + 1

    # Zona Peligro Indice
    resultado_final = pd.read_csv('C:/Users/20391117579/Dropbox/CrimeApp/Datasets/Zona peligro/zona_puntaje.csv')
    def danger_zone(zona):
        return resultado_final.loc[resultado_final['Zona'] == zona, 'Puntaje Normalizado'].iloc[0]
    
    df_fct_time_series['ZONA_PELIGRO_INDICE'] = df_fct_time_series['ZONA'].apply(danger_zone)

    df_modelado = df_fct_time_series

    return df_modelado
    

In [17]:
df_modelado = transform_location(df_fct_time_series, kmeans, scaler)

In [18]:
df_modelado

,FECHA,LATITUD,LONGITUD,ESTACION,MES,TRIMESTRE,SEMANA_DEL_AÑO,DIA_DEL_AÑO,DIA_DEL_MES,DIA_DE_LA_SEMANA,ES_FIN_DE_SEMANA,HORARIO_CATEGORIZADO,FRANJA_HORARIA,COMUNA_KEY,BARRIO_KEY,TIPO_DELITO_KEY,CLUSTER,ZONA,ZONA_PELIGRO_INDICE
0,2016-01-31,-34.6398,-58.5297,4,1,1,4,31,31,6,1,3,12,9,22,1,6,7,0.926473
1,2016-01-31,-34.6388,-58.5275,4,1,1,4,31,31,6,1,2,8,9,22,1,6,7,0.926473
2,2016-01-31,-34.6456,-58.5265,4,1,1,4,31,31,6,1,4,20,9,22,1,6,7,0.926473
3,2016-01-31,-34.6013,-58.5199,4,1,1,4,31,31,6,1,4,22,11,31,1,1,2,0.928730
4,2016-01-31,-34.5896,-58.5187,4,1,1,4,31,31,6,1,2,8,11,31,1,1,2,0.928730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066646,2023-12-01,-34.6174,-58.4976,4,12,4,48,335,1,4,0,1,3,10,26,1,6,7,0.926473
1066647,2023-12-01,-34.6273,-58.4563,4,12,4,48,335,1,4,0,2,8,7,17,3,5,6,0.945760
1066648,2023-12-01,-34.6312,-58.4316,4,12,4,48,335,1,4,0,2,7,7,18,1,0,1,0.988577
1066649,2023-12-01,-34.6363,-58.4495,4,12,4,48,335,1,4,0,2,8,7,18,1,5,6,0.945760
